k-NN (k-Nearest Neighbors) – algorytm k-najbliższych sąsiadów:

-> Algorytm uczenia maszynowego do klasyfikacji, który przewiduje klasę nowej próbki na podstawie k najbliższych obserwacji ze zbioru uczącego.

-> Metoda train zapisuje przypadki uczące, przyjmując wektory cech (dane) oraz klasy (etykiety).

Wielokrotne wywołanie train powinno rozszerzać zbiór uczący, a nie go nadpisywać.

-> Metoda predict przyjmuje wektor (lub zbiór wektorów, jako dane) i zwraca odpowiedź klasyfikatora.

Działanie algorytmu:

    1. Oblicza odległości nowej próbki od wszystkich przykładów w zbiorze uczącym.
    2. Wybiera k najbliższych sąsiadów.
    3. Przypisuje klasę, która występuje najczęściej wśród tych k sąsiadów.

Mierzenie odległości jest możliwe z użyciem metryki: euklidesowej, taksówkowej, maksimum i cosinusowej.

-> Algorytm nie wymaga trenowania w klasycznym sensie – jedynie zapamiętuje dane uczące i wykorzystuje je do klasyfikacji w czasie rzeczywistym.

In [1]:
import csv
import numpy as np
from enum import Enum

class DistanceMetric(Enum):
    EUCLIDEAN = 'euclidean'
    MANHATTAN = 'manhattan'
    CHEBYSHEV = 'chebyshev'
    COSINE = 'cosine'

class KNNClassifier:
    def __init__(self, k=3, distance_metric=DistanceMetric.EUCLIDEAN):
        if not isinstance(distance_metric, DistanceMetric):
            raise ValueError(f"Invalid distance metric. Choose from: {list(DistanceMetric)}")
        
        self.k = k
        self.distance_metric = distance_metric
        self.x_train = np.empty((0,))
        self.y_train = np.empty((0,), dtype=int)

    def train(self, x, y):
        x, y = np.array(x), np.array(y)
        if self.x_train.size == 0:
            self.x_train, self.y_train = x, y
        else:
            self.x_train = np.vstack((self.x_train, x))
            self.y_train = np.hstack((self.y_train, y))

    def _compute_distances(self, x):
        if self.distance_metric == DistanceMetric.EUCLIDEAN:
            return np.linalg.norm(self.x_train - x, axis=1)
        
        elif self.distance_metric == DistanceMetric.MANHATTAN:
            return np.sum(np.abs(self.x_train - x), axis=1)
        
        elif self.distance_metric == DistanceMetric.CHEBYSHEV:
            return np.max(np.abs(self.x_train - x), axis=1)
        
        elif self.distance_metric == DistanceMetric.COSINE:
            x_norm = np.linalg.norm(x)
            train_norms = np.linalg.norm(self.x_train, axis=1)
            ratio = np.dot(self.x_train, x) / (train_norms * x_norm)
            return 1 - ratio
        
        else:
            raise ValueError("Unsupported distance metric")

    def predict(self, X):
        X = np.atleast_2d(X)  # Zapewniamy, że X zawsze jest 2D
        if X.shape[1] != self.x_train.shape[1]:
            raise ValueError(f"Input dimension {X.shape[1]} does not match training data dimension {self.x_train.shape[1]}")

        predictions = []
        for x in X:
            distances = self._compute_distances(x)
            k_nearest_index = np.argpartition(distances, self.k)[:self.k]
            labels = self.y_train[k_nearest_index]
            predictions.append(np.argmax(np.bincount(labels)))

        return np.array(predictions) if len(predictions) > 1 else predictions[0]

In [2]:
def open_csv(file_path):
    x, y = [], []
    with open(file_path, 'r') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=' ')
        for row in csvreader:
            x.append(list(map(float, row[:-1])))
            y.append(int(row[-1]))
    return np.array(x), np.array(y)

In [3]:
x, y = open_csv('dataset0.csv')       
print(x)
print(y)
# Testowanie klasyfikatora
knn = KNNClassifier()
knn.train(x, y)
new_data = np.array([1.0,2.0])
prediction = knn.predict(new_data)
print(f'Prediction for new data {new_data} is {prediction}') # 1

knn.train(np.array([1., 2.]), np.array(1))
new_data = np.array([14.0, 23.0])
prediction = knn.predict(new_data)
print(f'Prediction for new data {new_data} is {prediction}') # 1

[[0. 0.]
 [0. 1.]
 [1. 1.]
 [1. 0.]]
[0 1 2 1]
Prediction for new data [1. 2.] is 1
Prediction for new data [14. 23.] is 1


In [4]:
x, y = open_csv('dataset0.csv')       
# Testowanie klasyfikatora
knn = KNNClassifier()
knn.train(x, y)
new_data = np.array([[1.0,2.0], [1.0,3.0]])
prediction = knn.predict(new_data)
print(f'Prediction for new data {new_data} is {prediction}') # 1 1

Prediction for new data [[1. 2.]
 [1. 3.]] is [1 1]


In [5]:
x, y = open_csv('dataset1.csv')
# Testowanie klasyfikatora
knn = KNNClassifier(distance_metric=DistanceMetric.CHEBYSHEV)
knn.train(x, y)
prediction = knn.predict(np.array([121.314024, 222.530757]))
print(f'Prediction: {prediction}') # 1

knn.train(np.array([121.314024, 222.530757]), np.array(1))
prediction = knn.predict(np.array([0.45540533, 6.420678]))
print(f'Prediction: {prediction}') # 0

Prediction: 1
Prediction: 0


In [6]:
x, y = open_csv('dataset2.csv')
print(x)
print(y)
print(np.array([121.314024, 222.530757, 23.3578, 87.4542]).size)
# Testowanie klasyfikatora
knn = KNNClassifier(distance_metric=DistanceMetric.MANHATTAN)
knn.train(x, y)
prediction = knn.predict(np.array([121.314024, 222.530757, 23.3578, 87.4542]))
print(f'Prediction: {prediction}') # 1

knn.train(np.array([121.314024, 222.530757, 23.3578, 87.4542]), np.array(1))
prediction = knn.predict(np.array([0.45540533, 6.420678, 86.3435, 2.3546]))
print(f'Prediction: {prediction}') # 0

[[15.73094725  7.18528284 64.41038317 24.66428237]
 [83.44155279 63.01982687 75.83614336 67.61127283]
 [99.39100517  9.61315971 14.93347999 72.15498409]
 ...
 [87.97905359 25.80428991 22.24284486 88.04449276]
 [80.20238287 21.64052602 41.48732363  3.6147353 ]
 [ 0.62492591 99.65803104 81.97150773 77.52295713]]
[0 1 1 ... 1 0 0]
4
Prediction: 1
Prediction: 0


In [7]:
x, y = open_csv('dataset2.csv')
# Testowanie klasyfikatora
knn = KNNClassifier(distance_metric=DistanceMetric.EUCLIDEAN, k=5)
knn.train(x, y)
prediction = knn.predict(np.array([121.314024, 222.530757, 23.3578, 87.4542]))
print(f'Prediction: {prediction}') # 1

knn.train(np.array([121.314024, 222.530757, 23.3578, 87.4542]), np.array(1))
prediction = knn.predict(np.array([0.45540533, 6.420678, 86.3435, 2.3546]))
print(f'Prediction: {prediction}') # 0

Prediction: 1
Prediction: 0
